In [ ]:
import numpy as np         # библиотека для матриц и математики
import pandas as pd        # библиотека для работы с табличками
from scipy import stats    # модуль для работы со статистикой

# библиотеки для визуализации
import matplotlib.pyplot as plt
import seaborn as sns

import math

from statsmodels.stats.weightstats import ztest as ztest

plt.style.use('ggplot')    # стиль графиков

%matplotlib inline


# 8.1 Расчёт парных коэффициентов корреляции

In [ ]:
#A7 A8
df = pd.read_excel("/content/drive/MyDrive/Study MePHI/сем5/matstat/HW/data_matstat_K5.xls", sheet_name="A - aaup")

## доверительные интервалы для линейного коэффициента корреляции

In [ ]:
std1 = df['A7'].std()
std2 = df['A8'].std()
mean1 = df.A7.mean()
mean2 = df.A8.mean()
n = df.A8.shape[0]

sample1 = df['A7'].to_numpy()
sample2 = df['A8'].to_numpy()

s = 0
for i in range(0, n):
  s += (sample1[i] - mean1) * (sample2[i] - mean2)
k = s / n

r = k / (std1 * std2)

alpha = 0.1

quantile = stats.norm(loc = 0, scale = 1).ppf(1 - (alpha/2))

a = r + (r*(1-r**2)/(2*n)) - ((1-r**2)/n**(1/2)) * quantile
b = r + (r*(1-r**2)/(2*n)) + ((1-r**2)/n**(1/2)) * quantile

print(a)
print(b)


0.9203648087787327
0.9344295077300105


0.9273366407860514

In [ ]:
n

1073

## Проверить значимость коэффициентов корреляции

In [ ]:
std1 = df['A7'].std()
std2 = df['A8'].std()
mean1 = df.A7.mean()
mean2 = df.A8.mean()
n = df.A8.shape[0]

sample1 = df['A7'].to_numpy()
sample2 = df['A8'].to_numpy()

s = 0
for i in range(0, n):
  s += (sample1[i] - mean1) * (sample2[i] - mean2)
k = s / n

r = k / (std1 * std2)

alpha = 0.1

z = (r / (1 - r**2)**(1/2)) * (n-2)**(1/2)
f_z = stats.t(n-2).cdf(z)
p_value = 2 * min(f_z, 1 - f_z)

print("test1: z = %.3f; p-value = %.3f" % (z, p_value))

test1: z = 81.095; p-value = 0.000


In [86]:
s1 = pd.Series(sample1)
s2 = pd.Series(sample2)


sp = 0
for i in range(0, n):
  sp += (s1.rank()[i] - s2.rank()[i])**2
sp = 1 - (6 / (n * (n**2 - 1))) * sp

z = (sp / (1 - sp**2)**(1/2)) * (n-2)**(1/2)

f_z = stats.t(n-2).cdf(z)
p_value = 2 * min(f_z, 1 - f_z)

print("test2: z = %.3f; p-value = %.3f" % (z, p_value))

test2: z = 88.288; p-value = 0.000


In [104]:
ken = stats.kendalltau(sample1, sample2)

z = ken[0] * ((9*n*(n+1))/(2*(2*n+5)))**(1/2)
f_z = stats.norm(loc = 1, scale = 1).cdf(z)
p_value = 2 * min(f_z, 1 - f_z)

print("test3: z = %.3f; p-value = %.3f" % (z, p_value))

test3: z = 38.636; p-value = 0.000


In [89]:
#KK

r = stats.pearsonr(sample1, sample2)
p = stats.spearmanr(sample1, sample2)   #спирмен
t = stats.kendalltau(sample1, sample2)  #кендалл
print(r)
print(p)
print(t)
type(t[0])

(0.9282016936226041, 0.0)
SpearmanrResult(correlation=0.9376540833607301, pvalue=0.0)
KendalltauResult(correlation=0.7868780458733714, pvalue=0.0)


numpy.float64

# 8.2. Расчёт множественных коэффициентов корреляции

In [ ]:
#A5 A7 A8

sample1 = df['A5'].to_numpy()
sample2 = df['A7'].to_numpy()
sample3 = df['A8'].to_numpy()

n1 = len(sample1)
n2 = len(sample2)
n3 = len(sample3)

In [107]:
# матрица ранговых коэффициентов корреляции по Кендаллу
# матрица значений p-value для ранговых коэффициентов корреляции по Кендаллу

sample = [sample1, sample2, sample3]
arr_kendall = []
arr_kendall_pvalue = []

for i in range(0,3):
  row = []
  row_p = []
  for j in range(0,3):
    if i != j:
      t = stats.kendalltau(sample[i], sample[j])
      row.append(t[0])
      row_p.append(t[1])
    else:
      t = stats.kendalltau(sample[i], sample[j])
      row.append(t[0])
      row_p.append('Null')
  arr_kendall.append(row)
  arr_kendall_pvalue.append(row_p)

arr_kendall_pvalue

[['Null', 0.0, 0.0], [0.0, 'Null', 0.0], [0.0, 0.0, 'Null']]

In [109]:
#точечная оценка коэффициента конкордации

s1 = pd.Series(sample1)
s2 = pd.Series(sample2)
s3 = pd.Series(sample3)

r1 = s1.rank()
r2 = s2.rank()
r3 = s3.rank()

r = [r1, r2, r3]

n = n1
k = 3       #число выборок

w = 0
for i in range(0, n):
  s1 = 0
  for j in range(0, k):
    s1 += (1/k) * r[j][i]
  w += (s1 - (n+1)/2)**2

w = (w * 12) / (n**3 - n)

print('Коэффициент конкордации: %.3f' % w)

#статистика критерия

z = k * (n - 1) * w
f_z = stats.chi2(n-1).cdf(z)
p_value = 2 * min(f_z, 1 - f_z)

print("test: z = %.3f; p-value = %.3f" % (z, p_value))

Коэффициент конкордации: 0.972
test: z = 3126.793; p-value = 0.000
